In [1]:
import pandas as pd
import docx
import redis
import pyarrow as pa

In [2]:
def parse_document(doc_path, skill_path, tier_start=-1):
    doc = docx.Document(doc_path)
    paras = [x.text for x in doc.paragraphs]
    for i in range(len(paras)):
        if paras[i] == ' ':
            paras[i] = ''
    particular_value = ''
    result = []
    temp_list = []
    for i in paras:
        if i == particular_value:
            temp_list.append(i)
            result.append(temp_list)
            temp_list = []
        else:
            temp_list.append(i)
    result.append(temp_list)
    df = pd.DataFrame({'text':result})
    tier_list = []
    ability_list = []
    tier = tier_start
    for _, row in df.iterrows():
        data = row['text']
        if len(data) > 1:
            if data[0].split(' ')[0] == 'Tier':
                tier += 1
                data.pop(0)
            tier_list.append(tier)
            ability_list.append(data)
    ability_df = pd.DataFrame({'Ability':ability_list, 'Tier':tier_list})
    ability_df['Path'] = skill_path
    ability_df['Skill Name'] = ability_df.Ability.apply(lambda x:x[0].split(':')[0])
    try:
        ability_df['Description'] = ability_df.Ability.apply(lambda x:x[0].split(':')[1])
    except:
        print(ability_df.Ability)
    pr = []
    lim = []
    preq = []
    aug = []
    for _ , row in ability_df.iterrows():
        for sublist in row['Ability']:
            if 'Phys Rep' in sublist:
                pr.append(sublist.split(':')[1])
            if 'Limitations' in sublist:
                lim.append(sublist.split(':')[1])
            if 'Prerequisite' in sublist:
                preq.append(sublist.split(':')[1])
            if 'Augment' in sublist:
                aug.append(sublist.split(':')[1])
        if 'Phys Rep' not in str(row.Ability):
            pr.append(None)
        if 'Limitations' not in str(row.Ability):
            lim.append(None)
        if 'Prerequisite' not in str(row.Ability):
            preq.append(None)
        if 'Augment' not in str(row.Ability):
            aug.append(None)
    ability_df['Phys Rep'] = pr
    ability_df['Limitations'] = lim
    ability_df['Prerequisite'] = preq
    ability_df['Augment'] = aug
    ability_df = ability_df.map(lambda x: x.strip() if isinstance(x, str) else x)
    return ability_df[['Skill Name', 'Description', 'Path', 'Tier', 'Limitations', 'Phys Rep', 'Prerequisite', 'Augment']]

In [3]:
artificer_df = parse_document("The Artificer's Path.docx", 'Artificer', tier_start=0)

In [4]:
artificer_df

,Skill Name,Description,Path,Tier,Limitations,Phys Rep,Prerequisite,Augment
0,Appraise,"Upon taking the Appraise skill, an artificer c...",Artificer,1,"In terms of roleplay, your keen eye for object...","To Appraise - a magnifying glass, examiner’s l...",None,None
1,Tinkerer’s Quirk,You may attach a mechanism to an object that a...,Artificer,1,None,"an actual light emitting material, a picture, ...",None,None
2,Schematic Encryption,You may disguise the meaning of your Artificer...,Artificer,1,None,A Schematic (see The Object’s Schematics below...,None,None
3,Artificer’s Cooperative Action,The best Artificers learn that they can get fu...,Artificer,1,Keep in mind that if you encrypt a Schematic a...,All Artificers must be within touch distance o...,Schematic Encryption.,None
4,Artorias’ Silver Weapon,"Through careful art, science, and ritual, you ...",Artificer,1,None,A bladed weapon. A number of silver pieces th...,None,"Expending a Small gem and a Fusion Point, grou..."


In [5]:
warrior_df = parse_document("The Warrior's Path.docx", 'Warrior')
rogue_df = parse_document("The Rogue's Path.docx", 'Rogue')
healer_df = parse_document("The Healer's Path.docx", 'Healer')
mage_df = parse_document("The Mage's Path.docx", 'Mage')
bard_df = parse_document("The Bard's Path.docx", 'Bard', tier_start=0)
artificer_df = parse_document("The Artificer's Path.docx", 'Artificer', tier_start=0)

In [6]:
rogue_df

,Skill Name,Description,Path,Tier,Limitations,Phys Rep,Prerequisite,Augment
0,Loot,Player gains the ability to loot bodies left o...,Rogue,0,Players cannot [Loot] another player's persona...,Player holds hand over the location being loot...,None,None
1,Detect Trap,"Using this ability, Rogues can now see traps l...",Rogue,0,Each search is limited to a single item or are...,"Role play, 15 seconds. The player detecting tr...",None,None
2,Shield Training,Rogues gain the ability to safely use small to...,Rogue,0,None,None,None,None
3,Basic Weapon Proficiency,Players learn the basic combat and safety rule...,Rogue,0,None,None,None,None
4,Armor Proficiency,Players learn about the armor that they may we...,Rogue,0,None,None,None,None
5,Kindle Flame/Torch,Player gains proficiency at creating normal fi...,Rogue,0,10’ radius,Optional (larp safe electronically lighted tor...,None,None
6,Cooperative Action,Rogues find out early on that the best way to ...,Rogue,1,None,All rogues must be within touch distance of th...,Detect Trap,None
7,Pick Lock,The player is now able to pick locks of a stre...,Rogue,1,None,A skeleton key or lockpicking kit of a nature ...,None,None
8,Bypass Trap,"After [Detect Trap], Rogues may now choose to ...",Rogue,1,None,The Rogue should carry a set of thieves' tools...,Detect Trap,None
9,Climb Walls,The player can now scale walls and cliffs. Thi...,Rogue,1,Not all obstacles can be climbed for safety re...,"In order for this skill to be put into effect,...",None,None


In [7]:
skills = pd.concat([warrior_df,rogue_df,healer_df,mage_df,bard_df, artificer_df])

In [8]:
skills['Spell'] = skills['Description'].apply(lambda x: x.startswith(('(Spell)', '(Combat Magic)')))

In [9]:
skills

,Skill Name,Description,Path,Tier,Limitations,Phys Rep,Prerequisite,Augment,Spell
0,Basic Weapon Proficiency,Players learn the basic combat and safety rule...,Warrior,0,None,None,None,None,False
1,Armor Proficiency,Players learn about the armor that they wear a...,Warrior,0,None,None,None,None,False
2,Kindle Flame/Torch,Player gains proficiency at creating normal fi...,Warrior,0,10’ radius,Optional (larp safe electronically lighted tor...,None,None,False
3,Shield Control,Players learn the art of Shield Control and ma...,Warrior,0,None,None,None,None,False
4,Advanced Weapon Training,Study and become proficient in the use of two-...,Warrior,1,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...
0,Appraise,"Upon taking the Appraise skill, an artificer c...",Artificer,1,"In terms of roleplay, your keen eye for object...","To Appraise - a magnifying glass, examiner’s l...",None,None,False
1,Tinkerer’s Quirk,You may attach a mechanism to an object that a...,Artificer,1,None,"an actual light emitting material, a picture, ...",None,None,False
2,Schematic Encryption,You may disguise the meaning of your Artificer...,Artificer,1,None,A Schematic (see The Object’s Schematics below...,None,None,False
3,Artificer’s Cooperative Action,The best Artificers learn that they can get fu...,Artificer,1,Keep in mind that if you encrypt a Schematic a...,All Artificers must be within touch distance o...,Schematic Encryption.,None,False


In [10]:
skills[skills.Path == 'Mage']

,Skill Name,Description,Path,Tier,Limitations,Phys Rep,Prerequisite,Augment,Spell
0,Read/Write Arcana,Player can decipher magic runes and read magic...,Mage,0,"Study, research, and quests may be needed to u...",Rune cipher in spellbook (Elder Futhark is the...,None,None,False
1,Light,(Spell) Create light equal to 1 torch. Can cou...,Mage,0,10’ radius,10 word spell in spellbook. Optional (lighted ...,None,None,True
2,Darkness,(Spell) Counters magical light. May be used to...,Mage,0,10’ radius,"10 word spell in spellbook. Optional, black cl...",None,None,True
3,Sense Arcana,"Can sense if an object has arcane properties, ...",Mage,0,"Single person or object, may be used 3x per da...",10 word spell in spellbook.,None,None,False
4,Basic Weapon Proficiency,Players learn the basic combat and safety rule...,Mage,0,None,None,None,None,False
5,Kindle Flame/Torch,Player gains proficiency at creating normal fi...,Mage,0,10’ radius,LARP safe light source.,None,None,False
6,Minor Repair,(Spell) Repairs a single small normal object 1...,Mage,1,"Single person or object/touch, may be used 3x ...","25 word spell in spellbook, and roleplay.",None,None,True
7,Divine Arcana,(Spell) Once an object is defined as magical t...,Mage,1,1x per Mage Tier per day per day.,"25 word spell in spellbook, and roleplay.",Sense Arcana,None,True
8,Arcane Armor,(Spell) Absorbs the first 1 point of damage ta...,Mage,1,May be used 1x per day +1 per Mage Tier above ...,"25 word spell in spellbook, red belt flag with...",None,None,True
9,Ward Location,(Spell) Areas marked cannot be entered or exit...,Mage,1,"1x per Tier per day, 10’ radius per tier, a Ma...","25 word spell in spellbook, and roleplay. Yell...",None,None,True


In [11]:
skills.to_excel('Skills_Table.xlsx', index=False)

In [12]:
skills.to_csv('Skills_Table.csv', index=False)

In [13]:
from math import floor, sqrt
def tier(events):
    return floor((sqrt(8*events)-1)/2)

In [14]:
skills[skills['Skill Name']=='Basic Weapon Proficiency']

,Skill Name,Description,Path,Tier,Limitations,Phys Rep,Prerequisite,Augment,Spell
0,Basic Weapon Proficiency,Players learn the basic combat and safety rule...,Warrior,0,None,None,None,None,False
3,Basic Weapon Proficiency,Players learn the basic combat and safety rule...,Rogue,0,None,None,None,None,False
5,Basic Weapon Proficiency,Players learn the basic combat and safety rule...,Healer,0,None,None,None,None,False
4,Basic Weapon Proficiency,Players learn the basic combat and safety rule...,Mage,0,None,None,None,None,False
